# change to load data from load_data.py


data source: ./simplified_data/simplified_data2.csv

data source: ./simplified_data/simplified_data2.csv

data 處理:

data v2 刪掉所有文字input

deleted 回診,西藥

## train 方法:

.....



In [1]:

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.callbacks import EarlyStopping, LambdaCallback
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from sklearn.utils.class_weight import compute_sample_weight




In [2]:
import numpy as np
import pandas as pd
import random
import json
import statistics
from tabulate import tabulate

# custom import 
import my_utilities as myutil
import load_data

## Read Data

1. 決定是否要delete 少於某threshold的藥
2. 用load_data裡的`load_data_for_n_med` load data
3. 如只train 頭n 個藥-> set `only_train_1st_n`

In [3]:
del_med_under_thres = 0     # 於所有medical cases中出現次數少於此數的藥->整col 刪除
                            # if set to 250 -> will leave 10 medicine

total_med = 102
only_train_1st_n = total_med

(X_np, X_val_np, train_y, val_y, 
  num_col_x, num_1_valy, num_0_valy) = load_data.load_data_for_1_med_with_debug(del_med_thres=del_med_under_thres, n=only_train_1st_n)

assert(isinstance(X_np, np.ndarray))
assert(isinstance(X_val_np, np.ndarray))
assert(isinstance(train_y, pd.DataFrame))
assert(isinstance(val_y, pd.DataFrame))
assert(isinstance(num_col_x, int))


in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).
SplitXY:
Shape of X = (796 rows, 111 cols). First 10 data of X:
    肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤  肝炎
1    0    0   0   0     0   0    0   0   0   0
2    0    0   0   0     0   0    0   0   0   0
3    0    0   0   0     0   0    0   0   0   0
4    0    0   0   0     0   0    0   0   0   0
5    0    0   0   0     0   0    0   0   0   0
6    0    0   0   0     0   0    0   0   0   0
7    0    0   0   0     0   0    0   0   0   0
8    0    0   0   0     0   0    0   0   0   0
9    0    0   0   0     0   0    0   0   0   0
10   0    0   0   0     0   0    0   0   0   0
Shape of y = (796 rows, 102 cols). First 10 data of y:
    麻黃  桂枝  荊芥  防風  細辛  白芷  生薑  辛夷  葛根  升麻
1    0   1   0   0   1   1   0   0   0   0
2    0   1   0   0   0   0   1   0   0   0
3    0   0   0   0   0   0   0   0   0   0
4    0   1   0   0   0   0   1   1   0   0
5    0   1   0   0   0   0   0   0   0   0
6   

In [4]:
# data type checking  you can run this if you suspect data type 
# else can skip this cell


# myutil.print_df(val_y, "---- y ----")
# print(val_y)
# checking
# 1. Count occurrence of na, int, float, str in y
na_count = val_y.isna().sum().sum()
str_count = val_y[val_y.map(type) == str].count().sum()
int_count = val_y[val_y.map(type) == int].count().sum()
float_count = val_y[val_y.map(type) == float].count().sum()

print(f"Number of NA values in y: {na_count}")
print(f"Number of str values in y: {str_count}")
print(f"Number of int values in y: {int_count}")
print(f"Number of float values in y: {float_count}")

Number of NA values in y: 0
Number of str values in y: 0
Number of int values in y: 16218
Number of float values in y: 0


## compute class weight

In [5]:

# # class_weights = compute_sample_weight(class_weight='balanced', y=train_y)
# # class_weight_dict = dict(enumerate(class_weights))

# # print(class_weight_dict)

# train_y_np = np.array(train_y)

# num_labels = train_y_np.shape[1]
# class_weight_dic = {}

# for i in range(num_labels):
#     unique_values, counts = np.unique(train_y_np[:, i], return_counts=True)
#     value_frequency_dict = dict(zip(unique_values, counts))
#     total = value_frequency_dict[0] + value_frequency_dict[1]
#     class_weight_dic[i] = {0: (value_frequency_dict[1] / total), 1: 5 * (value_frequency_dict[0] / total)} 
#     # f1: 0.35
#     # class_weight_dic[i] = {0: (total / value_frequency_dict[0]), 1: 5 * (total / value_frequency_dict[1])}

# print(class_weight_dic)
# class_weight_list = np.array(class_weight_dic)

## Build Model

In [6]:
model = Sequential([
    Dense (units=64, input_shape=(num_col_x,), activation='sigmoid'),
    Dropout(0.1),
    Dense (units=128, activation='softmax'), 
    Dropout(0.1),
    Dense (units=64, activation='softmax'), 
    Dropout(0.1),
    Dense (units=32, activation='softmax'), 
    Dense (units=2, activation='softmax')
])
model.summary()

model.compile(optimizer=Adam(learning_rate=0.005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                7168      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 32)                2

## Train Model

train medicine-by-medicine

In [7]:
all_res=[]
result_df_dict={}   # dict of df
#loss_acc_dict={}    # dict of loss and accuracy of each medicine
accuracy_dict={}    # dict of accuracy of each medicine

for i in range(train_y.shape[1]):
#for i in range(2):
    res = {}
    res_list =[]
    chosen_col = train_y.iloc[:,i].copy()
    assert(isinstance(chosen_col, pd.Series))
    assert(len(chosen_col) == len(train_y))
    print("processing medicine:", chosen_col.name)
    chosen_y_np = chosen_col.values.astype('float64')

    y_val_chosen_col = val_y.iloc[:,i].copy()
    assert(isinstance(y_val_chosen_col, pd.Series))
    assert(len(y_val_chosen_col) == len(val_y))
    #y_val_chosen_col_np = y_val_chosen_col.values.astype('float64')

    
    # Early stop
    early_stopping = EarlyStopping(monitor='loss', patience=100, restore_best_weights=True)
    
    # fit model for this medicine
    model.fit(
        x=X_np,
        y=chosen_y_np,
        # class_weight=class_weight_dic[i],
        epochs=2000,
        shuffle=True,
        verbose=0,
        callbacks=[early_stopping]
    )
    
    # make prediction for this medicine
    predictions = model.predict(X_val_np)
    print(type(predictions))
    print("predictions.shape:", predictions.shape)
    
    df_predictions = pd.DataFrame(predictions, columns=["predicted as 0", "predicted as 1"])
    df_predictions["predicted value"] = np.where(df_predictions["predicted as 0"] > df_predictions["predicted as 1"], 0, 1)
    df_predictions["ground truth"] = y_val_chosen_col.values
     # create new column that checks if "predicted value" is equal to "ground truth"
    df_predictions["is_correct"] = df_predictions["predicted value"] == df_predictions["ground truth"]

    accuracy = df_predictions["is_correct"].mean()
    accuracy_dict[chosen_col.name] = accuracy  
    result_df_dict[chosen_col.name] = df_predictions
    

print("training done.")




processing medicine: 麻黃
5/5 [==============================] - 0s 751us/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 桂枝
5/5 [==============================] - 0s 631us/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 荊芥
5/5 [==============================] - 0s 2ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 防風
5/5 [==============================] - 0s 0s/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 細辛
5/5 [==============================] - 0s 1ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 白芷
5/5 [==============================] - 0s 500us/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 生薑
5/5 [==============================] - 0s 1ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 辛夷
5/5 [==============================] - 0s 750us/step
<class 'numpy.ndarray'>
predi

## Handle result

In [14]:

predicted_value_index = None
ground_truth_index = None

acc_each_med={}
total_tp=0
total_fp=0
total_fn=0
total_tn=0

# df.columns.tolist()= pick list of column names

for i in train_y.columns.tolist(): # build empty dict for holding TP, FP, FN, TN, accuracy, f1_score of each medicine
    acc_each_med[i]={"TP": None, "FP": None, "FN": None, "TN": None, "accuracy": None, "f1_score": None}

# calculate f1 score of for each medicine

for key, df in result_df_dict.items():
    # create 'TP/FP/TN/TN/FN' column

    if predicted_value_index is None:
        predicted_value_index = df.columns.get_loc('predicted value')
    if ground_truth_index is None:
        ground_truth_index = df.columns.get_loc('ground truth')


    df['TP/FP/TN/TN/FN'] = df.apply(lambda row: 'TP' if ( row.iloc[predicted_value_index] and row.iloc[ground_truth_index] ) else 
                                 ('FP' if (row.iloc[predicted_value_index]  and (not row.iloc[ground_truth_index] )) else 
                                  ('FN' if (( not row.iloc[predicted_value_index] ) and row.iloc[ground_truth_index] ) else 
                                   ( 'TN' if ((not row.iloc[predicted_value_index]) and (not row.iloc[ground_truth_index]) ) else 'UN'))), axis=1)

    # count occurrences of 'TP', 'FP', 'FN', and 'TN' of *this medicine*
    counts = df['TP/FP/TN/TN/FN'].value_counts()
    # get number of 'TP', 'FP', 'FN', and 'TN'
    num_tp = counts.get('TP', 0)
    num_fp = counts.get('FP', 0)
    num_fn = counts.get('FN', 0)
    num_tn = counts.get('TN', 0)

    f1_score = 2 * num_tp / (2 * num_tp + num_fp + num_fn) if (2 * num_tp + num_fp + num_fn) > 0 else 0
    acc_each_med[key]["TP"] = num_tp
    acc_each_med[key]["FP"] = num_fp
    acc_each_med[key]["FN"] = num_fn
    acc_each_med[key]["TN"] = num_tn
    acc_each_med[key]["accuracy"] = accuracy_dict[key].item()
    acc_each_med[key]["f1_score"] = f1_score

    total_tp += num_tp
    total_fp += num_fp
    total_fn += num_fn
    total_tn += num_tn

    ### debug messages
    print("in for loop of result_df_dict")
    print(f"processing : {key} ....")
    print(f"TP of {key}: {num_tp}", sep="\t")
    print(f"FP of {key}: {num_fp}", sep="\t")
    print(f"FN of {key}: {num_fn}", sep="\t")
    print(f"TN of {key}: {num_tn}", sep="\t")
    print(f"f1_score of {key}: {f1_score}")
    print(f"total tp now is: {total_tp}"  , sep="\t")
    print(f"total fp now is: {total_fp}"  , sep="\t")
    print(f"total fn now is: {total_fn}"  , sep="\t")
    print(f"total tn now is: {total_tn}"  , sep="\t")

overall_f1 = 2 * total_tp / (2 * total_tp + total_fp + total_fn) if (2 * total_tp + total_fp + total_fn) > 0 else 0
total_med_case =  len(val_y)* only_train_1st_n
print(" \n*****   end of for loop   *****\n")
print("now doing calculation checking...")
assert((total_tp + total_fp + total_fn + total_tn) == total_med_case)
assert((total_tp + total_fn)== num_1_valy)  # TP+FN=all 1 in val_y
assert((total_fp + total_tn)== num_0_valy)  # TN+FP=all 0 in val_y
print("checking passed")

in for loop of result_df_dict
processing : 麻黃 ....
TP of 麻黃: 3
FP of 麻黃: 7
FN of 麻黃: 23
TN of 麻黃: 126
f1_score of 麻黃: 0.16666666666666666
total tp now is: 3
total fp now is: 7
total fn now is: 23
total tn now is: 126
in for loop of result_df_dict
processing : 桂枝 ....
TP of 桂枝: 26
FP of 桂枝: 18
FN of 桂枝: 44
TN of 桂枝: 71
f1_score of 桂枝: 0.45614035087719296
total tp now is: 29
total fp now is: 25
total fn now is: 67
total tn now is: 197
in for loop of result_df_dict
processing : 荊芥 ....
TP of 荊芥: 0
FP of 荊芥: 2
FN of 荊芥: 8
TN of 荊芥: 149
f1_score of 荊芥: 0.0
total tp now is: 29
total fp now is: 27
total fn now is: 75
total tn now is: 346
in for loop of result_df_dict
processing : 防風 ....
TP of 防風: 0
FP of 防風: 5
FN of 防風: 14
TN of 防風: 140
f1_score of 防風: 0.0
total tp now is: 29
total fp now is: 32
total fn now is: 89
total tn now is: 486
in for loop of result_df_dict
processing : 細辛 ....
TP of 細辛: 18
FP of 細辛: 26
FN of 細辛: 39
TN of 細辛: 76
f1_score of 細辛: 0.3564356435643564
total tp now is: 47


In [15]:
mean_accuracy = statistics.mean(accuracy_dict.values())

print(f"total tp: {total_tp}, \t percentage = {total_tp/total_med_case}" )
print(f"total fp: {total_fp}, \t percentage = {total_fp/total_med_case}" )
print(f"total fn: {total_fn}, \t percentage = {total_fn/total_med_case}" )
print(f"total tn: {total_tn}, \t percentage = {total_tn/total_med_case}" )
print(f"overall f1 score: {overall_f1}")
print("mean accuracy of all medicine: ", mean_accuracy)
print(acc_each_med)         # can't use json.dumps as there are np.int64



total tp: 532, 	 percentage = 0.03280305833025034
total fp: 782, 	 percentage = 0.04821802935010482
total fn: 1358, 	 percentage = 0.08373412257984955
total tn: 13546, 	 percentage = 0.8352447897397953
overall f1 score: 0.33208489388264667
mean accuracy of all medicine:  0.8680478480700456
{'麻黃': {'TP': 3, 'FP': 7, 'FN': 23, 'TN': 126, 'accuracy': 0.8113207547169812, 'f1_score': 0.16666666666666666}, '桂枝': {'TP': 26, 'FP': 18, 'FN': 44, 'TN': 71, 'accuracy': 0.610062893081761, 'f1_score': 0.45614035087719296}, '荊芥': {'TP': 0, 'FP': 2, 'FN': 8, 'TN': 149, 'accuracy': 0.9371069182389937, 'f1_score': 0.0}, '防風': {'TP': 0, 'FP': 5, 'FN': 14, 'TN': 140, 'accuracy': 0.8805031446540881, 'f1_score': 0.0}, '細辛': {'TP': 18, 'FP': 26, 'FN': 39, 'TN': 76, 'accuracy': 0.5911949685534591, 'f1_score': 0.3564356435643564}, '白芷': {'TP': 0, 'FP': 1, 'FN': 2, 'TN': 156, 'accuracy': 0.9811320754716981, 'f1_score': 0.0}, '生薑': {'TP': 12, 'FP': 16, 'FN': 33, 'TN': 98, 'accuracy': 0.6918238993710691, 'f1_sco

In [16]:

file_path_suffix = "each_med_csv"   # type the dir for you to remember where u save the result
for key, df in result_df_dict.items():
    print(f"DataFrame for {key}:")
    myutil.print_df(df)
    myutil.df_to_csv(df, save_path=("./result/"+file_path_suffix), file_prefix=key)
   



DataFrame for 麻黃:

+-----+------------------+------------------+-------------------+----------------+--------------+------------------+
|     |   predicted as 0 |   predicted as 1 |   predicted value |   ground truth | is_correct   | TP/FP/TN/TN/FN   |
|-----+------------------+------------------+-------------------+----------------+--------------+------------------|
|   0 |           0.9620 |           0.0380 |                 0 |              0 | True         | TN               |
|   1 |           0.9620 |           0.0380 |                 0 |              0 | True         | TN               |
|   2 |           0.9613 |           0.0387 |                 0 |              0 | True         | TN               |
|   3 |           0.7834 |           0.2166 |                 0 |              0 | True         | TN               |
|   4 |           0.9620 |           0.0380 |                 0 |              0 | True         | TN               |
|   5 |           0.9620 |           0.0380 |

In [ ]:
# save f1_score and TP/FP/TN/TN/FN

spec_str = "debug, epoch=10, batch_size=50, learning_rate=0.01, only train first 3 medicine"
# need to type this spec str each time to record the result

file_path="./result/1_med_accuracy"
myutil.dict_to_txt(acc_each_med, save_path=file_path, 
                   file_prefix="accuracy_each_med",
                   textbox=spec_str)

accuracy_each_med saved to ./result/1_med_accuracy/accuracy_each_med_56.txt
